In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math
import warnings
warnings.filterwarnings("ignore")

# Load the dataset
file_path = 'common_records_freddy.csv'
data = pd.read_csv(file_path)

# Function to create sequences for CNN-LSTM model
def create_sequences(data, seq_length=24):
    sequences = []
    targets = []
    for i in range(len(data) - seq_length):
        seq = data.iloc[i:i+seq_length][['Demand_kWh', 'Temperature']].values
        target = data.iloc[i+seq_length]['Demand_kWh']
        sequences.append(seq)
        targets.append(target)
    return np.array(sequences), np.array(targets)

# Define CNN-LSTM model
def create_cnn_lstm_model(input_shape):
    model = Sequential([
        Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=input_shape),
        MaxPooling1D(pool_size=2),
        LSTM(50, activation='relu', return_sequences=True),
        LSTM(50, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# Preprocess the data
def preprocess_data(data):
    # Ensure the data is sorted by Date
    data = data.sort_values(by=['From']).reset_index(drop=True)
    
    # Normalize the temperature columns only
    temp_columns = ['Temperature']
    # data[temp_columns] = (data[temp_columns] - data[temp_columns].mean()) / data[temp_columns].std()

    return data

# Split the data into train and test sets
def split_data(data, test_size=0.5):
    train_data = {}
    test_data = {}

    ids = data['ID'].unique()
    
    for id_ in ids:
        id_data = data[data['ID'] == id_]
        train, test = train_test_split(id_data, test_size=test_size, shuffle=False)
        train_data[id_] = train 
        test_data[id_] = test
    
    return train_data, test_data

# Preprocess the dataset
data = preprocess_data(data)

# Split the data into train and test sets
train_data, test_data = split_data(data)

# Initialize lists to store results
results = []
k =1
# Train and evaluate model for each ID
for id_ in train_data.keys():  # Corrected slicing
    # Prepare training and testing data
    print(f"{k} Processing ID: {id_}")
    k =k+1
    train_sequences, train_targets = create_sequences(train_data[id_])
    test_sequences, test_targets = create_sequences(test_data[id_])
    
    # Reshape input data for the CNN-LSTM model
    input_shape = (train_sequences.shape[1], train_sequences.shape[2])
    
    # Create and train the model
    model = create_cnn_lstm_model(input_shape)
    model.fit(train_sequences, train_targets, epochs=50, batch_size=32, validation_split=0.2, verbose=0)
    
    # Predict on the test data
    predictions = model.predict(test_sequences)
    
    # Store results
    for i in range(len(test_targets)):
        results.append([id_, test_data[id_].iloc[i + 24]['From'], test_targets[i], predictions[i][0]])

# Save predictions to a CSV file
results_df = pd.DataFrame(results, columns=['ID', 'Date', 'Actual_Demand', 'Predicted_Demand'])
results_df.to_csv('common_records_method1_cnn_lstm_5050.csv', index=False)

# Calculate overall metrics
overall_mse = mean_squared_error(results_df['Actual_Demand'], results_df['Predicted_Demand'])
overall_mae = mean_absolute_error(results_df['Actual_Demand'], results_df['Predicted_Demand'])
overall_rmse = math.sqrt(overall_mse)

print(f"Overall Test Loss (MSE): {overall_mse}")
print(f"Overall Test Loss (MAE): {overall_mae}")
print(f"Overall Test Loss (RMSE): {overall_rmse}")


1 Processing ID: Exp_98
71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step
2 Processing ID: Exp_5
71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step
3 Processing ID: Exp_707
71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step
4 Processing ID: Exp_164
71/71 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step
5 Processing ID: Exp_171
71/71 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step
6 Processing ID: Exp_541
71/71 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step
7 Processing ID: Exp_611
71/71 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step
8 Processing ID: Exp_108
71/71 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step
9 Processing ID: Exp_144
71/71 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step
10 Processing ID: Exp_190
71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step
11 Processing ID: Exp_560
71/71 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step
12 Processing ID: Exp_50
71/71 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step
13 Processing ID: Exp_479
71/71 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step
14 Processing ID: Exp_514
71/71 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step
15 Processing ID: Exp_185
71/71 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step
16 Processing ID: Exp_1

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math
import warnings
warnings.filterwarnings("ignore")

# Load the dataset
file_path = 'common_records_freddy.csv'
data = pd.read_csv(file_path)

# Function to create sequences for CNN-LSTM model
def create_sequences(data, seq_length=24):
    sequences = []
    targets = []
    for i in range(len(data) - seq_length):
        seq = data.iloc[i:i+seq_length][['Demand_kWh', 'Temperature']].values
        target = data.iloc[i+seq_length]['Demand_kWh']
        sequences.append(seq)
        targets.append(target)
    return np.array(sequences), np.array(targets)

# Define CNN-LSTM model
def create_cnn_lstm_model(input_shape):
    model = Sequential([
        Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=input_shape),
        MaxPooling1D(pool_size=2),
        LSTM(50, activation='relu', return_sequences=True),
        LSTM(50, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# Preprocess the data
def preprocess_data(data):
    # Ensure the data is sorted by Date
    data = data.sort_values(by=['From']).reset_index(drop=True)
    
    # Normalize the temperature columns only
    temp_columns = ['Temperature']
    # data[temp_columns] = (data[temp_columns] - data[temp_columns].mean()) / data[temp_columns].std()

    return data

# Split the data into train and test sets
def split_data(data, test_size=0.25):
    train_data = {}
    test_data = {}

    ids = data['ID'].unique()
    
    for id_ in ids:
        id_data = data[data['ID'] == id_]
        train, test = train_test_split(id_data, test_size=test_size, shuffle=False)
        train_data[id_] = train 
        test_data[id_] = test
    
    return train_data, test_data

# Preprocess the dataset
data = preprocess_data(data)

# Split the data into train and test sets
train_data, test_data = split_data(data)

# Initialize lists to store results
results = []
k =1
# Train and evaluate model for each ID
for id_ in train_data.keys():  # Corrected slicing
    # Prepare training and testing data
    print(f"{k} Processing ID: {id_}")
    k =k+1
    train_sequences, train_targets = create_sequences(train_data[id_])
    test_sequences, test_targets = create_sequences(test_data[id_])
    
    # Reshape input data for the CNN-LSTM model
    input_shape = (train_sequences.shape[1], train_sequences.shape[2])
    
    # Create and train the model
    model = create_cnn_lstm_model(input_shape)
    model.fit(train_sequences, train_targets, epochs=50, batch_size=32, validation_split=0.2, verbose=0)
    
    # Predict on the test data
    predictions = model.predict(test_sequences)
    
    # Store results
    for i in range(len(test_targets)):
        results.append([id_, test_data[id_].iloc[i + 24]['From'], test_targets[i], predictions[i][0]])

# Save predictions to a CSV file
results_df = pd.DataFrame(results, columns=['ID', 'Date', 'Actual_Demand', 'Predicted_Demand'])
results_df.to_csv('common_records_method1_cnn_lstm_7525.csv', index=False)

# Calculate overall metrics
overall_mse = mean_squared_error(results_df['Actual_Demand'], results_df['Predicted_Demand'])
overall_mae = mean_absolute_error(results_df['Actual_Demand'], results_df['Predicted_Demand'])
overall_rmse = math.sqrt(overall_mse)

print(f"Overall Test Loss (MSE): {overall_mse}")
print(f"Overall Test Loss (MAE): {overall_mae}")
print(f"Overall Test Loss (RMSE): {overall_rmse}")


1 Processing ID: Exp_98
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step
2 Processing ID: Exp_5
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step
3 Processing ID: Exp_707
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step
4 Processing ID: Exp_164
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step
5 Processing ID: Exp_171
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step
6 Processing ID: Exp_541
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step
7 Processing ID: Exp_611
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step
8 Processing ID: Exp_108
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step
9 Processing ID: Exp_144
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step
10 Processing ID: Exp_190
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step
11 Processing ID: Exp_560
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step
12 Processing ID: Exp_50
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step
13 Processing ID: Exp_479
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step
14 Processing ID: Exp_514
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step
15 Processing ID: Exp_185
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 46ms/step
16 Processing ID: Exp_1

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math
import warnings
warnings.filterwarnings("ignore")

# Load the dataset
file_path = 'common_records_freddy.csv'
data = pd.read_csv(file_path)

# Function to create sequences for CNN-LSTM model
def create_sequences(data, seq_length=24):
    sequences = []
    targets = []
    for i in range(len(data) - seq_length):
        seq = data.iloc[i:i+seq_length][['Demand_kWh', 'Temperature']].values
        target = data.iloc[i+seq_length]['Demand_kWh']
        sequences.append(seq)
        targets.append(target)
    return np.array(sequences), np.array(targets)

# Define CNN-LSTM model
def create_cnn_lstm_model(input_shape):
    model = Sequential([
        Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=input_shape),
        MaxPooling1D(pool_size=2),
        LSTM(50, activation='relu', return_sequences=True),
        LSTM(50, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# Preprocess the data
def preprocess_data(data):
    # Ensure the data is sorted by Date
    data = data.sort_values(by=['From']).reset_index(drop=True)
    
    # Normalize the temperature columns only
    temp_columns = ['Temperature']
    # data[temp_columns] = (data[temp_columns] - data[temp_columns].mean()) / data[temp_columns].std()

    return data

# Split the data into train and test sets
def split_data(data, test_size=0.2):
    train_data = {}
    test_data = {}

    ids = data['ID'].unique()
    
    for id_ in ids:
        id_data = data[data['ID'] == id_]
        train, test = train_test_split(id_data, test_size=test_size, shuffle=False)
        train_data[id_] = train 
        test_data[id_] = test
    
    return train_data, test_data

# Preprocess the dataset
data = preprocess_data(data)

# Split the data into train and test sets
train_data, test_data = split_data(data)

# Initialize lists to store results
results = []
k =1
# Train and evaluate model for each ID
for id_ in train_data.keys():  # Corrected slicing
    # Prepare training and testing data
    print(f"{k} Processing ID: {id_}")
    k =k+1
    train_sequences, train_targets = create_sequences(train_data[id_])
    test_sequences, test_targets = create_sequences(test_data[id_])
    
    # Reshape input data for the CNN-LSTM model
    input_shape = (train_sequences.shape[1], train_sequences.shape[2])
    
    # Create and train the model
    model = create_cnn_lstm_model(input_shape)
    model.fit(train_sequences, train_targets, epochs=50, batch_size=32, validation_split=0.2, verbose=0)
    
    # Predict on the test data
    predictions = model.predict(test_sequences)
    
    # Store results
    for i in range(len(test_targets)):
        results.append([id_, test_data[id_].iloc[i + 24]['From'], test_targets[i], predictions[i][0]])

# Save predictions to a CSV file
results_df = pd.DataFrame(results, columns=['ID', 'Date', 'Actual_Demand', 'Predicted_Demand'])
results_df.to_csv('common_records_method1_cnn_lstm_8020.csv', index=False)

# Calculate overall metrics
overall_mse = mean_squared_error(results_df['Actual_Demand'], results_df['Predicted_Demand'])
overall_mae = mean_absolute_error(results_df['Actual_Demand'], results_df['Predicted_Demand'])
overall_rmse = math.sqrt(overall_mse)

print(f"Overall Test Loss (MSE): {overall_mse}")
print(f"Overall Test Loss (MAE): {overall_mae}")
print(f"Overall Test Loss (RMSE): {overall_rmse}")

1 Processing ID: Exp_98
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step
2 Processing ID: Exp_5
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step
3 Processing ID: Exp_707
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step
4 Processing ID: Exp_164
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step
5 Processing ID: Exp_171
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step
6 Processing ID: Exp_541
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step
7 Processing ID: Exp_611
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step
8 Processing ID: Exp_108
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step
9 Processing ID: Exp_144
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step
10 Processing ID: Exp_190
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step
11 Processing ID: Exp_560
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step
12 Processing ID: Exp_50
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step
13 Processing ID: Exp_479
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step
14 Processing ID: Exp_514
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step
15 Processing ID: Exp_185
28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step
16 Processing ID: Exp_1